In [1]:
from torchvision.datasets import FashionMNIST
from torchvision import transforms
from torch.utils.data import DataLoader


# 訓練用のデータ取得
# transforms.ToTensor()で画像をTensorに変換する
fashion_mnist_train = FashionMNIST("./FashionMNIST", train=True, download=True, transform=transforms.ToTensor())
# テスト洋データを取得
# transforms.ToTensor()で画像をTensorに変換する
fashion_mnist_test = FashionMNIST("./FashionMNIST", train=False, download=True, transform=transforms.ToTensor())
batch_size = 128
# バッチサイズが128のDataLoaderを作成する
train_loader = DataLoader(fashion_mnist_train, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(fashion_mnist_test, batch_size=batch_size, shuffle=True)

In [2]:
import torch
from torch import nn
from torch.autograd import Variable as V


# (N, C, H, W)を(N, C*H*W)に変換するレイヤー
# 畳み込みレイヤーの出力をmlpに渡すのに必要
# N：バッチサイズ（一度の計算でまとめて処理するデータ数）
# C：チャネル数（色数）
# H：画像の縦幅
# W：画像の横幅
class FlattenLayer(nn.Module):
    def forward(self, x):
        sizes = x.size()
        return x.view(sizes[0], -1)

In [3]:
# ２層の畳み込みレイヤー
# Conv2d(in_channels, out_channels, kernel_size, stride=1, padding=0, dilation=1, groups=1, bias=True)
# MaxPool2d(kernel_size, stride=None, padding=0, dilation=1, return_indices=False, ceil_mode=False)

# Conv2d: 画像の畳込みレイヤー、畳み込みのフィルタを移動させることで特徴マップを作る
# MaxPool2d: 最大プーリングレイヤー、プーリング領域内の最大値を取り出して出力し特徴マップを簡素化する
# BatchNorm2d: 画像用バッチノーマリゼーション
# Dropout2d: 画像用dropout

# kernel: 畳み込みのフィルタやプーリング領域といった小領域のことを、まとめてカーネルと呼ぶ
# stride: カーネルの動く際のステップ
# padding: 入力の周りに「枠」を付けることで出力サイズを調整する
# dilation: カーネルにあける隙間の大きさ
conv_net = nn.Sequential(
    nn.Conv2d(1, 32, 5),
    nn.MaxPool2d(2),
    nn.ReLU(),
    nn.BatchNorm2d(32),
    nn.Dropout2d(0.25),
    nn.Conv2d(32, 64, 5),
    nn.MaxPool2d(2),
    nn.ReLU(),
    nn.BatchNorm2d(64),
    nn.Dropout2d(0.25),
    FlattenLayer()
)

In [4]:
# 試しに畳み込みレイヤーに入力してみる
test_input = V(torch.ones(1, 1, 28, 28))
conv_output_size = conv_net(test_input).size()
print(conv_output_size)
conv_output_size = conv_output_size[-1]

torch.Size([1, 1024])


In [5]:
# ２層のmlp
mlp = nn.Sequential(
    nn.Linear(conv_output_size, 200),
    nn.ReLU(),
    nn.BatchNorm1d(200),
    nn.Dropout(0.25),
    nn.Linear(200, 10)
)

In [6]:
# ２層の畳み込みレイヤーと２層のmlpをつなげてCNNを構成
net = nn.Sequential(
    conv_net,
    mlp
)

In [7]:
def eval_net(net, data_loader):
    # ネットワークを評価モードにする(dropoutやバッチノーマリゼーションを無効化する)
    net.eval()
    ys = []
    ypreds = []
    for x, y in data_loader:
        x = V(x)
        y = V(y)
        # 確率が最大のクラスを取得
        _, y_pred = net(x).max(1)
        ys.append(y)
        ypreds.append(y_pred)
    # ミニバッチごとの正解と予測結果を一つにまとめる
    ys = torch.cat(ys)
    ypreds = torch.cat(ypreds)
    # 予測精度を計算
    acc = (ys == ypreds).float().sum() / len(ys)
    return acc.data[0]

In [8]:
from torch import optim
from matplotlib import pyplot as plt
from tqdm import tqdm


def train_net(net, train_loader, test_loader, optimizer_cls=optim.Adam, loss_fn=nn.CrossEntropyLoss(), n_iter=10):
    train_losses = []
    train_acc = []
    val_acc = []
    optimizer = optimizer_cls(net.parameters())
    for epoch in range(n_iter):
        running_loss = 0.0
        # ネットワークを訓練モードにする(dropoutやバッチノーマリゼーションを有効化する)
        net.train()
        n = 0
        n_acc = 0
        for i, (x, y) in tqdm(enumerate(train_loader), total=len(train_loader)):
            xx = V(x)
            yy = V(y)
            h = net(xx)
            loss = loss_fn(h, yy)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            running_loss += loss.data[0]
            n += len(xx)
            _, y_pred = h.max(1)
            n_acc += (yy == y_pred).float().sum().data[0]
        train_losses.append(running_loss / i)
        train_acc.append(n_acc / n)
        val_acc.append(eval_net(net, test_loader))
        print(epoch, train_losses[-1], train_acc[-1], val_acc[-1], flush=True)

In [9]:
train_result = train_net(net, train_loader, test_loader, n_iter=20)

100%|██████████| 469/469 [00:45<00:00, 10.24it/s]


0 0.4744661207127775 0.8359 0.8744999766349792


100%|██████████| 469/469 [00:49<00:00,  9.57it/s]


1 0.3170749327311149 0.8848666666666667 0.8934000134468079


100%|██████████| 469/469 [00:46<00:00, 10.15it/s]


...

17 0.15759160614803305 0.9417333333333333 0.9211000204086304


100%|██████████| 469/469 [00:45<00:00, 10.20it/s]


18 0.1544505790926707 0.94245 0.9175000190734863


100%|██████████| 469/469 [00:46<00:00, 10.05it/s]


19 0.1499001835910683 0.9441 0.9205999970436096
